In [5]:
import os
import re
import pandas as pd

In [6]:

current_path = os.getcwd()

file_to_explore = "/full_data.csv"
file_path = current_path + file_to_explore

full_data = pd.read_csv(file_path)
print(full_data)

       Unnamed: 0   fix_id                                     modified_files  \
0               0   740304  ['toolkit/crashreporter/client/crashreporter_o...   
1               1  1191862  ['widget/WidgetEventImpl.cpp', 'widget/cocoa/n...   
2               2  1362483                                                 []   
3               3  1363276  ['dom/media/platforms/android/RemoteDataDecode...   
4               4  1399651         ['browser/base/content/browser-places.js']   
...           ...      ...                                                ...   
12467       12467  1806167  ['layout/reftests/xul/accesskey-ref.xhtml', 'l...   
12468       12468  1806199                  ['tools/lint/test/test_isort.py']   
12469       12469  1806844           ['gfx/thebes/gfxMacPlatformFontList.mm']   
12470       12470  1806944  ['testing/web-platform/meta/css/css-images/ima...   
12471       12471  1807697                   ['browser/modules/webrtcUI.jsm']   

                           

# How many bugs in the dataset have either bug file or file reference empty?

In [7]:
import json
# The requirement of NLP-szz is that there should be atlest one common file shared between bug intro and bug fix.
non_empty_referenced_files = full_data.loc[full_data['referenced_files']!='[]', 'fix_id']
non_empty_bug_files =  full_data.loc[full_data['bug_files']!='[]', 'fix_id']
common_non_empty_referenced_files = set(non_empty_bug_files).intersection(set(non_empty_referenced_files))
diff_non_empty_referenced_files = set(non_empty_bug_files).difference(set(non_empty_referenced_files))
print("reference files:",len(set(non_empty_referenced_files)))
print("bug files:",len(set(non_empty_bug_files)))
print("bug and reference files:",len(common_non_empty_referenced_files), ",uncommon:",len(diff_non_empty_referenced_files))

file_path_filtered_sample = "./Datasets/Filtered/RelevantSample/sample_filtered_normal_dataset.json"
file_path_filtered = "./Datasets/Filtered/filtered_normal_dataset.json"

with open(file_path_filtered, "r") as file:
    data_filtered = json.load(file)

fix_ids_filtered = [entry["id"] for entry in data_filtered]
print("filtered dataset: ",len(fix_ids_filtered))


with open(file_path_filtered_sample, "r") as file:
    data_filtered_sample = json.load(file)

fix_ids_filtered_sample = [entry["id"] for entry in data_filtered_sample]
print("filtered dataset sample:",len(fix_ids_filtered_sample))

common_non_empty_referenced_files_filtered = set(fix_ids_filtered_sample).intersection(set(fix_ids_filtered))
print("common_non_empty_referenced_files_filtered:",len(common_non_empty_referenced_files_filtered))



reference files: 9281
bug files: 11857
bug and reference files: 8919 ,uncommon: 2938
filtered dataset:  8299
filtered dataset sample: 620
common_non_empty_referenced_files_filtered: 0


# How many Bugs (manual analyzed) share references with bug-fix commits (Tangled), bug-introducing commits (Extrinsic) or none (no reference).

In [38]:
# Load the data
manual_analysis_path = "/RQ1/Manual_Analysis.xlsx"
manual_analysis_data = pd.read_excel(manual_analysis_path)

# Extract the 'fix_id' column
manual_fix_ids = manual_analysis_data['fix_id']
print(len(manual_fix_ids))

grouped_manual_fix_ids = manual_fix_ids.groupby(manual_analysis_data['Sheet'])
for group, fix_ids in grouped_manual_fix_ids:
    print("Group:", group)
    print("Fix IDs:", len(fix_ids.tolist()))
    print()


369
Group: FBC (Extrinsic)
Fix IDs: 106

Group: FFC (Tangled)
Fix IDs: 152

Group: FNC (NoRef)
Fix IDs: 111



# How many bugs in manual analysis have references that are not shared with any

In [39]:
fnc_noref_fix_ids = manual_analysis_data.loc[manual_analysis_data['Sheet'] == 'FNC (NoRef)', 'fix_id'].tolist()
print(len(fnc_noref_fix_ids))

ffc_tangled_fix_ids = manual_analysis_data.loc[manual_analysis_data['Sheet'] == 'FFC (Tangled)', 'fix_id'].tolist()
print(len(ffc_tangled_fix_ids))

fbc_extrinsic_fix_ids = manual_analysis_data.loc[manual_analysis_data['Sheet'] == 'FBC (Extrinsic)', 'fix_id'].tolist()
print(len(fbc_extrinsic_fix_ids))

111
152
106


# How many bugs have file references

In [13]:
# bugs with file references in their discussions
referenced = full_data['has_references'].value_counts()
referenced_norm = full_data['has_references'].value_counts(normalize=True)
referenced_df = pd.concat([referenced, referenced_norm], axis=1)
referenced_df.columns = ['count', 'norm']
referenced_df['cum_perc'] = referenced_df['norm'].cumsum()*100
print(referenced_df)

                count      norm    cum_perc
has_references                             
True             9281  0.744147   74.414689
False            3191  0.255853  100.000000


# How many bugs have valuable references?

In [12]:
 # A reference is valuable if it appears in either the bug or the fix commits
valuable_referenced = full_data['has_valuable_reference'].value_counts()
valuable_referenced_norm = full_data['has_valuable_reference'].value_counts(normalize=True)
valuable_referenced_df = pd.concat([valuable_referenced, valuable_referenced_norm], axis=1)
valuable_referenced_df.columns = ['count', 'norm']
valuable_referenced_df['cum_perc'] = valuable_referenced_df['norm'].cumsum()*100
print(valuable_referenced_df)

                        count      norm   cum_perc
has_valuable_reference                            
True                     6550  0.525176   52.51764
False                    5922  0.474824  100.00000


# How many bugs in manual analysis sample are invaluable as the references in their bug discussion are not shared with bug introducting and bug fixing bugs.

In [40]:
fix_ids_with_references = full_data.loc[full_data['has_references'], 'fix_id']
print("bug has reference:",len(fix_ids_with_references))
fix_ids_valuable_reference = full_data.loc[full_data['has_valuable_reference'] == True, 'fix_id']
print("bug has valuable reference:",len(fix_ids_valuable_reference))
fix_ids_not_valuable_reference = full_data.loc[full_data['has_valuable_reference'] == False, 'fix_id']
#print("bug has either not valuable reference or no reference:",len(fix_ids_not_valuable_reference))

count_fix_ids = (set(fix_ids_with_references).intersection(set(fix_ids_not_valuable_reference)))
print("invaluable references:",len(count_fix_ids))

# find the tangled bugs that are common in fix_ids_with_references
common_tangled_fix_ids = (set(ffc_tangled_fix_ids).intersection(set(count_fix_ids)))
print("tangled:",len(common_tangled_fix_ids))

# find the extrinsic bugs that are common in fix_ids_with_references
common_extrinsic_fix_ids = (set(fbc_extrinsic_fix_ids).intersection(set(count_fix_ids)))
print("extrinsic:",len(common_extrinsic_fix_ids))

# find the no reference bugs that are common with invaluable references
common_no_ref_fix_ids = (set(fnc_noref_fix_ids).intersection(set(count_fix_ids)))
print("fnc_noref_fix_ids:",len(fnc_noref_fix_ids))

already_fix_analyzed = {1743807, 1728063, 1547237, 1793766, 1759631, 1647735, 1676417, 1755242, 1783959, 1686750, 1603512, 1772523, 1578951, 1719963, 1640051, 1777952, 1749013, 1673164, 1685773, 1764690, 1673341, 1717833, 1631276, 1707595, 1703625, 1635783, 1637973, 1673680, 1651928, 1570684, 1668265, 1733707, 1645377, 1772489, 1570816, 1578750, 1570786, 1605347, 1661454, 1560720, 1556360, 1691901, 1596761, 1669096, 1637837, 1708729, 1585158, 1723198, 1701785, 1674756, 1607895, 1685744, 1664065, 1635481, 1582963, 1587794, 1710598, 1800323, 1581934, 1637564, 1600445, 1541660, 1733957, 1564241, 1682919, 1668706, 1586603, 1601905, 1761691, 1755006, 1641510, 1640906, 1648577, 1652232, 1688999, 1565370, 1595284, 1655544, 1719391, 1749500, 1719189, 1658550, 1747941, 1660213, 1751564}
print("Count of sample already analyzed", (set(already_fix_analyzed).intersection(set(fnc_noref_fix_ids))))

bug has reference: 9281
bug has valuable reference: 6550
invaluable references: 2731
tangled: 0
extrinsic: 0
fnc_noref_fix_ids: 111
Count of sample already analyzed set()


# Bugs whose file references are shared with bug introducing, bug fixing, both.

In [41]:
 # parsed_bug_counter: counter of files referenced in the issue parsed into buggy files
files_bug_intro =  full_data.loc[(full_data['parsed_bug_counter'] > 0), 'fix_id']
print("Bugs with file references shared with bug intro:",len(files_bug_intro))
files_only_bug_intro = full_data.loc[(full_data['parsed_counter'] == 0) & (full_data['parsed_bug_counter'] > 0), 'fix_id']
print("Bugs with file references shared with only bug intro:",len(files_only_bug_intro))

 # parsed_counter: counter of files referenced in the issue parsed in modified files
files_bug_fix =  full_data.loc[(full_data['parsed_counter'] > 0), 'fix_id']
print("Bugs with file references shared with bug fix:",len(files_bug_fix))
files_only_bug_fix = full_data.loc[(full_data['parsed_counter'] > 0) & (full_data['parsed_bug_counter'] == 0), 'fix_id']
print("Bugs with file references shared with only bug fix:",len(files_only_bug_fix))

 # both: counter of files referenced in the issue parsed in modified files
files_both_bug_fix = full_data.loc[(full_data['parsed_counter'] > 0) & (full_data['parsed_bug_counter'] > 0), 'fix_id']
print("Bugs with file references shared with both bug fix and bug intro:",len(files_both_bug_fix))

# see if ffc_tangled_fix_ids are also in files_bug_intro
common_tangled_files_bug_intro = (set(ffc_tangled_fix_ids).intersection(set(files_both_bug_fix)))
print("tangled bugs sharing with bug intro as well:",len(common_tangled_files_bug_intro))

# see if fbc_extrinsic_fix_ids are also in files_bug_intro
common_extrnisic_files_bug_intro = (set(fbc_extrinsic_fix_ids).intersection(set(files_both_bug_fix)))
print("extrinsic bugs sharing with bug fix as well:",len(common_extrnisic_files_bug_intro))

# check if already_fix_analyzed shared with files_bug_intro, files_bug_fix, files_both_bug_fix
already_fix_analyzed_shared_with_files_bug_intro = already_fix_analyzed.intersection(set(files_only_bug_intro))
already_fix_analyzed_shared_with_files_bug_fix = already_fix_analyzed.intersection(set(files_only_bug_fix))
already_fix_analyzed_shared_with_files_both_bug_fix = already_fix_analyzed.intersection(set(files_both_bug_fix))

print("already_fix_analyzed shared with files_bug_intro:", len(already_fix_analyzed_shared_with_files_bug_intro))
print("already_fix_analyzed shared with files_bug_fix:", len(already_fix_analyzed_shared_with_files_bug_fix))
print("already_fix_analyzed shared with files_both_bug_fix:", len(already_fix_analyzed_shared_with_files_both_bug_fix))


Bugs with file references shared with bug intro: 5555
Bugs with file references shared with only bug intro: 1179
Bugs with file references shared with bug fix: 5371
Bugs with file references shared with only bug fix: 995
Bugs with file references shared with both bug fix and bug intro: 4376
tangled bugs sharing with bug intro as well: 115
extrinsic bugs sharing with bug fix as well: 57
already_fix_analyzed shared with files_bug_intro: 0
already_fix_analyzed shared with files_bug_fix: 13
already_fix_analyzed shared with files_both_bug_fix: 72


# How many references are there in the manual analysis and how many of them are parsed and shared with commits?

In [51]:
# BEGIN: filter_rows
filtered_manual_analysis_sample_full = full_data[full_data['fix_id'].isin(manual_fix_ids)]
# END: filter_rows
print("filtered sample data from full dataset: ",len(manual_fix_ids))

filtered_reference_counter = filtered_manual_analysis_sample_full[filtered_manual_analysis_sample_full['parsed_counter'] != 0]['fix_id']
print("non empty references:",len(filtered_reference_counter))

reference_counter_sum = filtered_manual_analysis_sample_full['reference_counter'].sum()
print("referenced files in manual analysis sample:",reference_counter_sum)

parsed_references_counter_sum = filtered_manual_analysis_sample_full['parsed_counter'].sum()
print("parsed referenced files in manual analysis sample::",parsed_references_counter_sum)

filtered sample data from full dataset:  369
non empty references: 209
referenced files in manual analysis sample: 2482
parsed referenced files in manual analysis sample:: 304


# How many extrinsic_bug in dataset and manual analysis set

In [133]:
# when there are no common files shared between the files modified in bug fix and in the bug intro
extrinsic_bug = full_data.loc[full_data['extrinsic_bug'] == True, 'fix_id']
print("No of extrinsic_bug in dataset: ",len(extrinsic_bug))

extrinsic_reference = full_data.loc[full_data['extrinsic_reference'] == True, 'fix_id']
print("The bugs can be targeted by references:",len(extrinsic_reference))

manual_analyzed_extrinsic_ids = set(fbc_extrinsic_fix_ids).intersection(set(extrinsic_bug))
print("Manually analyzed extrinsic bugs:",len(manual_analyzed_extrinsic_ids))

No of extrinsic_bug:  4918
The bugs can be targeted by references: 1436
Manually analyzed extrinsic bugs: 15


# How many detangled bugs in overall dataset and manual analysis dataset?

In [135]:
# the fix has some files discussed in the bug ticket AND NOT(the fix has the same file discussed in the bug ticket)
detangled_bugs = full_data.loc[full_data['reduced_noise'] == True, 'fix_id']
print("No of detangled_bugs: ",len(detangled_bugs))

manual_analyzed_detangled_ids = set(ffc_tangled_fix_ids).intersection(set(detangled_bugs))
print("Manually analyzed extrinsic bugs:",len(manual_analyzed_detangled_ids))


No of detangled_bugs:  2164
Manually analyzed extrinsic bugs: 34


# How many manual analysis bugs that do not have valuable references?

In [137]:
fix_ids_with_valuable_reference = full_data.loc[full_data['has_valuable_reference'], 'fix_id']
print(len(fix_ids_with_valuable_reference))

fix_ids_false_valuable_reference = full_data.loc[full_data['has_valuable_reference'] == False, 'fix_id']
print(len(fix_ids_false_valuable_reference))

#Verify if the FFC (Tangled) are in the list of false valuable references
common_tangled_fix_ids = pd.Series(list(set(ffc_tangled_fix_ids).intersection(set(fix_ids_with_valuable_reference))))
print(len(common_tangled_fix_ids))

#Verify if the FBC (Extrinsic) are in the list of false valuable references
common_extrinsic_fix_ids = pd.Series(list(set(fbc_extrinsic_fix_ids).intersection(set(fix_ids_with_valuable_reference))))
print(len(common_extrinsic_fix_ids))

#Verify if the FNC (NoRef) are in the list of false valuable references
common_no_ref_fix_ids = pd.Series(list(set(fnc_noref_fix_ids).intersection(set(fix_ids_false_valuable_reference))))
print("not valuablae referece:",len(common_no_ref_fix_ids))




6550
5922
66
43
not valuablae referece: 266


# How many bug tickets contains a valuable reference that can be parsed to Mozilla's file and used to refine SZZ?

In [222]:
valuable_repo_referenced = full_data['has_relaxed_repo_valuable_reference'].value_counts()
valuable_repo_referenced_norm = full_data['has_relaxed_repo_valuable_reference'].value_counts(normalize=True)
valuable_repo_referenced_df = pd.concat([valuable_repo_referenced, valuable_repo_referenced_norm], axis=1)
valuable_repo_referenced_df.columns = ['count', 'norm']
valuable_repo_referenced_df['cum_perc'] = valuable_referenced_df['norm'].cumsum()*100
print(valuable_repo_referenced_df)

       count      norm   cum_perc
False  11011  0.882858  100.00000
True    1461  0.117142   52.51764


# How many bugs can be linked by SZZ?

In [225]:
# the fix and the bug shares at least one file
linkable = full_data['is_linkable'].value_counts()
linkable_norm = full_data['is_linkable'].value_counts(normalize=True)
linkable_df = pd.concat([linkable, linkable_norm], axis=1)
linkable_df.columns = ['count', 'norm']
linkable_df['cum_perc'] = linkable_df['norm'].cumsum()*100
print(linkable_df)

       count      norm    cum_perc
True    7554  0.605677   60.567672
False   4918  0.394323  100.000000


# How many bugs can be linked by SZZ adding file references?

In [226]:
linkable_ref = full_data['is_linkable_with_ref'].value_counts()
linkable_ref_norm = full_data['is_linkable_with_ref'].value_counts(normalize=True)
linkable_ref_df = pd.concat([linkable_ref, linkable_ref_norm], axis=1)
linkable_ref_df.columns = ['count', 'norm']
linkable_ref_df['cum_perc'] = linkable_ref_df['norm'].cumsum()*100
print(linkable_ref_df)

       count      norm    cum_perc
True    8990  0.720815   72.081462
False   3482  0.279185  100.000000


# How many bugs can be detangled?

In [228]:
# the fix has some files discussed in the bug ticket AND NOT(the fix has the same file discussed in the bug ticket)
noise_ref = full_data['reduced_noise'].value_counts()
noise_ref_norm = full_data['reduced_noise'].value_counts(normalize=True)
noise_ref_df = pd.concat([noise_ref, noise_ref_norm], axis=1)
noise_ref_df.columns = ['count', 'norm']
noise_ref_df['cum_perc'] = noise_ref_df['norm'].cumsum()*100
print(noise_ref_df)

       count      norm    cum_perc
False  10308  0.826491   82.649134
True    2164  0.173509  100.000000


# How many extrinsic bug can be detected by discussion files?

In [229]:
# the extrinsic bug is when the no file is shared between bug and fix commit
# the bug is extrinsic but there is a reference usable to link it as the file reference from bug discussion is shared with bugin using bug parsing or fix parsing strategy
bug_references = full_data['extrinsic_reference'].value_counts()
bug_references_norm = full_data['extrinsic_reference'].value_counts(normalize=True)
bug_references_df = pd.concat([bug_references, bug_references_norm], axis=1)
bug_references_df.columns = ['count', 'norm']
bug_references_df['cum_perc'] = bug_references_df['norm'].cumsum()*100
print(bug_references_df)

       count      norm    cum_perc
False  11036  0.884862   88.486209
True    1436  0.115138  100.000000


In [234]:
# How many references to fix files are in tickets?
ref_pd = full_data.loc[full_data['parsed_counter'] > 0]
print(len(ref_pd))
# How many references to bug files are in tickes?
bug_ref_pd = full_data.loc[full_data['parsed_bug_counter'] > 0]
print(len(bug_ref_pd))

5555


In [215]:
total_records = len(full_data['fix_id'])
print(total_records)

# Count the records where the modified files are empty
empty_modified_files = full_data[(full_data['modified_files'].isnull() | (full_data['modified_files'] == '[]'))]
print("empty_modified_files:",len(empty_modified_files))

 # referenced_files: file referenced in the bug discussion
empty_referenced_files = full_data[(full_data['referenced_files'].isnull() | (full_data['referenced_files'] == '[]'))]
print("empty_referenced_files:",len(empty_referenced_files))

# files associated to the bug
empty_bug_files = full_data[(full_data['bug_files'].isnull() | (full_data['bug_files'] == '[]'))]
print("empty_bug_files:",len(empty_bug_files))

 # bug_ids: ids of the bugs associated to the fix
empty_bugs = full_data[(full_data['bug_ids'].isnull() | (full_data['bug_files'] == '[]'))]
print("empty_bugs:",len(empty_bugs))


12472
empty_modified_files: 2383
empty_referenced_files: 3191
empty_bug_files: 615
empty_bugs: 615


In [244]:
# relaxed_repo_references: file referenced in the issue converted in a file of the repository based on path similarity with modified files
nonem_relaxed_repo_references = full_data[((full_data['relaxed_repo_references'] != '[]'))]
print("nonem_relaxed_repo_references:",len(nonem_relaxed_repo_references))

#fix_reference_shared: files shared between fix and discussion
ne_fix_reference_shared = full_data[(full_data['fix_reference_shared']> 0)]
print("ne_fix_reference_shared:",len(ne_fix_reference_shared))

#bug_reference_shared: # files shared between bug and discussion
ne_bug_reference_shared = full_data[(full_data['bug_reference_shared']> 0)]
print("ne_bug_reference_shared:",len(ne_bug_reference_shared))

# files shared between fix and discussion after fix parsing
ne_fix_reference_shared_after = full_data[(full_data['fix_fixsim_reference_shared']>0)]
print("ne_fix_reference_shared_after:",len(ne_fix_reference_shared_after))

# files shared between bug and discussion after fix parsing
ne_bug_fixsim_reference_shared = full_data[(full_data['bug_fixsim_reference_shared']>0)]
print("ne_bug_fixsim_reference_shared:",len(ne_bug_fixsim_reference_shared))

nonem_relaxed_repo_references: 5687
ne_fix_reference_shared: 1732
ne_bug_reference_shared: 1874
ne_fix_reference_shared_after: 5371
ne_bug_fixsim_reference_shared: 3782


# How many Sample bugs (used to run SZZ) have referenced files?

In [279]:
sample_dfp = "./Datasets/Filtered/RelevantSample/sample_filtered_normal_dataset.json"

with open(sample_dfp, "r") as file:
    sample_dfp_data = json.load(file)

fix_ids_dfp = [entry["id"] for entry in sample_dfp_data]
print("filtered dataset: ",len(fix_ids_dfp))

empty_fix_commits_ids = [entry['id'] for entry in sample_dfp_data if not entry['fix_commit_hash']]
print("bugs with empty fix commits:",len(empty_fix_commits_ids))

empty_files_ids = [entry['id'] for entry in sample_dfp_data if not entry['files']]
non_empty_files_ids = [entry['id'] for entry in sample_dfp_data if entry['files']]
print("bugs with empty files:",len(empty_files_ids))
print("bugs with non empty files:",len(non_empty_files_ids))

print("files and fix commit empty",len(set(empty_files_ids).intersection(set(empty_fix_commits_ids))))
 
parsed_files = sum(len(entry['files']) for entry in sample_dfp_data if entry['id'] in fix_ids_dfp)
print("Total number of files in fix_ids_dfp:", parsed_files)

# BEGIN: filter_rows
filtered_data_sampe_full = full_data[full_data['fix_id'].isin(non_empty_files_ids)]
# END: filter_rows
print("filtered sample data from full: ",len(filtered_data_sampe_full))

reference_counter_sum = filtered_data_sampe_full['reference_counter'].sum()
print("referenced files:",reference_counter_sum)

parsed_references_counter_sum = filtered_data_sampe_full['parsed_counter'].sum()
print("parsed referenced files:",parsed_references_counter_sum)



filtered dataset:  620
bugs with empty fix commits: 93
bugs with empty files: 93
bugs with non empty files: 527
files and fix commit empty 0
Total number of files in fix_ids_dfp: 1725
filtered sample data from full:  527
referenced files: 4175
parsed referenced files: 634


# How many tangled commits in sample szz data of 620 bugs

In [280]:
# the fix has some files discussed in the bug ticket AND NOT(the fix has the same file discussed in the bug ticket)
noise_ref_szz = filtered_data_sampe_full['reduced_noise'].value_counts()
noise_ref_szz_norm = filtered_data_sampe_full['reduced_noise'].value_counts(normalize=True)
noise_ref_df_szz = pd.concat([noise_ref_szz, noise_ref_szz_norm], axis=1)
noise_ref_df_szz.columns = ['count', 'norm']
noise_ref_df_szz['cum_perc'] = noise_ref_df_szz['norm'].cumsum()*100
print(noise_ref_df_szz)

               count      norm    cum_perc
reduced_noise                             
False            378  0.717268   71.726755
True             149  0.282732  100.000000


# How many extrinsic bug can be detected by discussion files in Sample szz data?

In [282]:
# the extrinsic bug is when the no file is shared between bug and fix commit
# the bug is extrinsic but there is a reference usable to link it as the file reference from bug discussion is shared with bugin using bug parsing or fix parsing strategy
bug_references_szz = filtered_data_sampe_full['extrinsic_reference'].value_counts()
bug_references_norm_szz = filtered_data_sampe_full['extrinsic_reference'].value_counts(normalize=True)
bug_references_df_szz = pd.concat([bug_references_szz, bug_references_norm_szz], axis=1)
bug_references_df_szz.columns = ['count', 'norm']
bug_references_df_szz['cum_perc'] = bug_references_df_szz['norm'].cumsum()*100
print(bug_references_df_szz)

                     count      norm    cum_perc
extrinsic_reference                             
False                  461  0.874763   87.476281
True                    66  0.125237  100.000000
